In [1]:
import torch
import math
import os
import sys
sys.path.append(r'../ncg_optimizer')
from fr import FR
#import ncg_optimizer as optim
MyDevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def quadratic(tensor):
    x, y = tensor
    a = 2.0
    b = 1.0
    return (x ** 2) / a + (y ** 2) / b + 2 * x * y - x
initial_state = (-2.0, -2.0)
min_loc = (-1, 1)
min = torch.tensor(quadratic(min_loc),device=MyDevice)

In [2]:
def rosenbrock(tensor):
    x, y = tensor
    return (1 - x) ** 2 + 1 * (y - x ** 2) ** 2
initial_state = (-2.0, -2.0)
min_loc = (1, 1)
min = torch.tensor(rosenbrock(min_loc),device=MyDevice)

In [2]:
def beale(tensor):
    x, y = tensor
    f = (
        (1.5 - x + x * y) ** 2
        + (2.25 - x + x * y ** 2) ** 2
        + (2.625 - x + x * y ** 3) ** 2
    )
    return f
initial_state = (1.5, 1.5)
min_loc = (3, 0.5)
min = torch.tensor(beale(min_loc),device=MyDevice)

In [2]:
def rastrigin(tensor, lib=torch):
    # https://en.wikipedia.org/wiki/Test_functions_for_optimization
    x, y = tensor
    A = 10
    f = (
        A * 2
        + (x ** 2 - A * lib.cos(x * math.pi * 2))
        + (y ** 2 - A * lib.cos(y * math.pi * 2))
    )
    return f
initial_state = (-2.0, 3.5)
min_loc = (0, 0)
min = torch.tensor(rastrigin(torch.tensor(min_loc,device=MyDevice)))

c:\Users\Administrator\.conda\envs\UUE\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [3]:
x = torch.tensor(initial_state,device=MyDevice).requires_grad_(True)
x_min = torch.tensor(min_loc,device=MyDevice)
iterations = 500

In [ ]:
#optimizer = optim.LCG([x], eps = 1e-3)
#optimizer = FR([x], eps=1e-3, line_search='Armijo', lr=0.1)
optimizer = FR([x], eps=1e-3, line_search='Wolfe', c2=0.9, lr=0.5, eta=5)
#optimizer = FR([x], eps=1e-3, line_search='None')
for _ in range(iterations):
    def closure():
        #print(x)
        optimizer.zero_grad()
        #f = quadratic(x)
        f = rosenbrock(x)
        #f = beale(x)
        #f = rastrigin(x)
        f.backward(retain_graph=True, create_graph=True)
        return f
    print(closure() - min)
    optimizer.step(closure)
    if torch.allclose(x, x_min.float(), atol=0.01) or torch.allclose(closure(), min.float(), atol=0.02):
        break
torch.allclose(closure(), min.float(), atol=0.02)